In [4]:
import torch

# Check if a GPU is available
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using GPU:", torch.cuda.get_device_name(0))  # 0 indicates the first GPU if you have multiple
else:
    device = torch.device("cpu")
    print("Using CPU")

Using GPU: NVIDIA A100-SXM4-80GB


In [3]:
import os
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from torchvision.io import read_image
import os
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from torchvision.io import read_image
from torchvision.transforms import ToTensor
from PIL import Image  # Import PIL's Image modul
import torch.optim as optim

In [8]:
!kaggle datasets download -d nih-chest-xrays/data

100%|█████████████████████████████████████▉| 42.0G/42.0G [10:02<00:00, 74.7MB/s]
100%|██████████████████████████████████████| 42.0G/42.0G [10:02<00:00, 74.9MB/s]


In [19]:
import os
import shutil

# Define the source directories
source_directories = ["images_003/images", "images_004/images",
                     "images_005/images", "images_007/images",
                     "images_001/images", "images_002/images",
                     "images_006/images", "images_008/images",
                     "images_009/images", "images_010/images",
                     "images_011/images", "images_012/images"]

# Define the destination directory
destination_directory = "image_folder/images"

# Create the destination directory if it doesn't exist
os.makedirs(destination_directory, exist_ok=True)

# Iterate through the source directories
for source_directory in source_directories:
    # List all files in the source directory
    files = os.listdir(source_directory)
    
    # Move or copy each file to the destination directory
    for file in files:
        source_path = os.path.join(source_directory, file)
        destination_path = os.path.join(destination_directory, file)
        # Use shutil.move() to move or shutil.copy() to copy
        shutil.move(source_path, destination_path)


In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms

# Define Swin Transformer Blocks
class SwinTransformerBlock(nn.Module):
    def __init__(self, dim, num_heads, window_size, mlp_ratio=4):
        super(SwinTransformerBlock, self).__init__()
        self.norm1 = nn.LayerNorm(dim)
        self.attention = nn.MultiheadAttention(dim, num_heads)
        self.norm2 = nn.LayerNorm(dim)
        self.mlp = nn.Sequential(
            nn.Linear(dim, mlp_ratio * dim),
            nn.GELU(),
            nn.Linear(mlp_ratio * dim, dim),
        )

    def forward(self, x):
        # Self-attention and MLP layers
        residual = x
        x = self.norm1(x)
        x = self.attention(x, x, x)[0] + residual
        residual = x
        x = self.norm2(x)
        x = self.mlp(x) + residual
        return x

# Define Swin Transformer
class SwinTransformer(nn.Module):
    def __init__(self, image_size, patch_size, num_classes, num_layers, num_heads, mlp_ratio=4):
        super(SwinTransformer, self).__init__()
        num_patches = (image_size // patch_size) ** 2
        dim = 96  # Adjust the dimension as needed for your task (e.g., 96, 192, 384)

        # Patch Embedding
        self.patch_embed = nn.Conv2d(1, dim, kernel_size=patch_size, stride=patch_size, padding=0)

        # Swin Transformer Blocks
        self.blocks = nn.ModuleList([
            SwinTransformerBlock(dim, num_heads, window_size=(image_size // patch_size), mlp_ratio=mlp_ratio)
            for _ in range(num_layers)
        ])

        # Classification Head
        self.norm = nn.LayerNorm(dim)
        self.fc = nn.Linear(dim, num_classes)

    def forward(self, x):
        # Extract patches and flatten
        x = self.patch_embed(x)
        x = x.flatten(2).transpose(1, 2)

        # Swin Transformer Blocks
        for block in self.blocks:
            x = block(x)

        # Global Average Pooling
        x = x.mean(dim=1)

        # Classification Head
        x = self.norm(x)
        x = self.fc(x)

        return x

# Example Usage:
image_size = 256  # Adjust as needed
patch_size = 4   # Adjust as needed
num_classes = 14  # Number of classes for multi-label classification
num_layers = 12   # Number of layers in the Swin Transformer
num_heads = 8     # Number of attention heads
mlp_ratio = 4     # MLP expansion ratio

model = SwinTransformer(image_size, patch_size, num_classes, num_layers, num_heads, mlp_ratio)


In [5]:
criterion = nn.BCEWithLogitsLoss()  # Replace with your loss function
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Replace with your optimizer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = model.to(device)

In [6]:
import os
import pandas as pd
from torchvision import transforms
from PIL import Image
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, csv_file, image_folder, transform=None):
        self.data = pd.read_csv(csv_file)
        self.image_folder = image_folder
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = os.path.join(self.image_folder, self.data.iloc[idx, 0])
        image = Image.open(img_name)

        # Check the number of channels in the image
        if image.mode == 'RGB':
            # Handle RGB-D images by converting them to grayscale
            image = image.convert('L')
        elif image.mode != 'L':
            # Handle other image modes (e.g., RGBA) by converting them to grayscale
            image = image.convert('L')

        labels = self.data.iloc[idx, 1:].values.astype('float32')  # Assuming labels start from the second column

        if self.transform:
            image = self.transform(image)

        return image, labels


In [7]:
import pandas as pd
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# Define the paths to your CSV files
train_csv = "train.csv"
val_csv = "validation.csv"
test_csv = "test.csv"

# Define the path to your image folder
image_folder_path = "image_folder/images"

# Define your data transformations
transform = transforms.Compose([  # Define your data transformations here
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize([0.485], [0.224]),
    # Add more transformations as needed
])

In [8]:

# Create custom datasets for each split
train_dataset = CustomDataset(train_csv, image_folder_path, transform=transform)
val_dataset = CustomDataset(val_csv, image_folder_path, transform=transform)
test_dataset = CustomDataset(test_csv, image_folder_path, transform=transform)

# Define batch sizes
batch_size = 32  # Adjust the batch size as needed

# Create DataLoader objects for each dataset
train_loader = DataLoader(train_dataset, batch_size=batch_size , shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size )


In [ ]:
import torch
import torch.nn as nn
from tqdm import tqdm
from sklearn.metrics import roc_auc_score

# Define the number of training and validation epochs
num_epochs = 50
num_runs = 1  # Number of times to run the training loop

# Lists to store training and validation loss for plotting
train_loss_history = []
val_loss_history = []

# Lists to store AUC values for train, val, and test
train_auc_history = []
val_auc_history = []
test_auc_history = []

# Outer loop for multiple runs
for run in range(num_runs):
    print(f"Run {run + 1}/{num_runs}")

    # Training and validation loop
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        running_train_loss = 0.0
        train_predictions = []
        train_labels = []

        train_loader_with_progress = tqdm(train_loader, desc=f"Epoch {epoch + 1}/{num_epochs}")

        for batch_idx, (inputs, labels) in enumerate(train_loader_with_progress):
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_train_loss += loss.item()

            train_predictions.extend(outputs.cpu().detach().numpy())
            train_labels.extend(labels.cpu().detach().numpy())

            # Print the training loss for each batch
            train_loader_with_progress.set_postfix({"Train Loss": loss.item()})

        average_train_loss = running_train_loss / len(train_loader)
        train_loss_history.append(average_train_loss)

        # Calculate AUC for training data
        train_auc = roc_auc_score(train_labels, train_predictions)
        train_auc_history.append(train_auc)

        # Validation phase
        model.eval()
        running_val_loss = 0.0
        val_predictions = []
        val_labels = []

        val_loader_with_progress = tqdm(val_loader, desc=f"Epoch {epoch + 1}/{num_epochs}")

        with torch.no_grad():
            for batch_idx, (inputs, labels) in enumerate(val_loader_with_progress):
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)

                running_val_loss += loss.item()

                val_predictions.extend(outputs.cpu().detach().numpy())
                val_labels.extend(labels.cpu().detach().numpy())

                # Print the validation loss for each batch during validation
                val_loader_with_progress.set_postfix({"Val Loss": loss.item()})

        average_val_loss = running_val_loss / len(val_loader)
        val_loss_history.append(average_val_loss)


        # Calculate AUC for validation data (your existing code)
        val_auc = roc_auc_score(val_labels, val_predictions)
        val_auc_history.append(val_auc)

        # Print the average training and validation loss for the epoch
        print(f"Run {run + 1}/{num_runs}, Epoch [{epoch + 1}/{num_epochs}] - Average Train Loss: {average_train_loss:.4f} - Average Val Loss: {average_val_loss:.4f} - Train AUC: {train_auc:.4f} - Val AUC: {val_auc:.4f}")


    # Calculate AUC for test data
    model.eval()
    test_predictions = []
    test_labels = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            test_predictions.extend(outputs.cpu().detach().numpy())
            test_labels.extend(labels.cpu().detach().numpy())

    test_auc = roc_auc_score(test_labels, test_predictions)
    test_auc_history.append(test_auc)
    print(f"Run {run + 1}/{num_runs} - Test AUC: {test_auc:.4f}")

    # Save your trained model if needed (you can save it with a unique name for each run)
    torch.save(model.state_dict(), f"model_full_extend{run + 1}.pth")

# Plot and save the AUC values
plt.figure(figsize=(8, 5))
plt.plot(range(1, num_runs + 1), test_auc_history, marker='o', linestyle='-')
plt.xlabel('Run')
plt.ylabel('Test AUC')
plt.title('Test AUC for Multiple Runs')
plt.grid(True)
plt.savefig('test_auc_plot_scrtatch.png')
plt.show()


Run 1/1


Epoch 1/50: 100%|██████████| 534/534 [30:58<00:00,  3.48s/it, Val Loss=0.104] 


Run 1/1, Epoch [1/50] - Average Train Loss: 0.1644 - Average Val Loss: 0.1642 - Train AUC: 0.5214 - Val AUC: 0.5590


Epoch 2/50: 100%|██████████| 534/534 [24:57<00:00,  2.80s/it, Val Loss=0.103] 


Run 1/1, Epoch [2/50] - Average Train Loss: 0.1629 - Average Val Loss: 0.1641 - Train AUC: 0.5359 - Val AUC: 0.5598


Epoch 3/50:  42%|████▏     | 915/2170 [56:46<1:06:39,  3.19s/it, Train Loss=0.138] 